In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.metrics import f1_score

In [3]:


def get_data(type="train"):
    data = []
    label = []
    for i in range(1, 37):
        X = []
        y = []
        target = f"{i:02d}"  
        train_data_path = f"../../data/q2/{type}/{target}"
        
        for img_name in os.listdir(train_data_path):
            img_path = os.path.join(train_data_path, img_name)
            
            
            img = cv2.imread(img_path)
            
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            
            img = cv2.resize(img, (32, 32))
            img = img / 255.0 
            img_flatten = img.flatten()
            X.append(img_flatten)
            y.append(i)

        data.extend(X)
        label.extend(y)

    data = np.array(data)
    label = np.array(label)
    return data, label

X_train , y_train = get_data("train")
X_test , y_test = get_data("test")
y_train  = np.eye(36)[y_train - 1]
y_test  = np.eye(36)[y_test - 1]


In [5]:


class Layer:
    def __init__(self,in_dimension , out_dimension , learning_rate = 0.001 , is_sigmoid = True,is_output = False):
        self.net_grad = None
        self.lr = learning_rate
        self.O = None
        limit = np.sqrt(6 / (in_dimension + out_dimension))
        self.w = np.random.uniform(-limit, limit, (in_dimension + 1, out_dimension))

        self.output_function = self.__sigmoid if is_sigmoid else self.__softmax
        self.is_output = is_output

    def __sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def __softmax(self,x):
        exp_x = np.exp(x - np.max(x , axis =1 , keepdims=True))
        return exp_x / np.sum(exp_x, axis = 1 , keepdims=True)

    def __sigmoid_grad(self,x):
        return x * (1-x)    

    def next(self,X ):
        X = np.hstack([np.ones((X.shape[0] , 1)),X])
        self.net_grad = np.dot(X, self.w)
        self.O = self.output_function(self.net_grad)
        return self.O

    def _prev(self, dO , O_down):
        if not self.is_output:
            new_grad = dO * self.__sigmoid_grad(self.O)
            dw = np.dot(np.hstack([np.ones((O_down.shape[0], 1)), O_down]).T , new_grad) / O_down.shape[0]
            d_O_down = np.dot(new_grad , self.w.T)
            dO_prev =   d_O_down[:,1:]
            self.w = self.w - self.lr * dw
            return  dO_prev
        
            
    

class FinalLayer :
    def __init__(self,in_dim , out_dim , lr = 0.001 ,is_softmax = True) :
        self.lr = lr
        limit = np.sqrt(6 / (in_dim + out_dim))
        self.w = np.random.uniform(-limit, limit, (in_dim + 1, out_dim))

        self.net_grad = None
        self.O = None
        self.output_function = self.__softmax if is_softmax else self.__sigmoid

    def __sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def __softmax(self,x):
        exp_x = np.exp(x - np.max(x , axis =1 , keepdims=True))
        return exp_x / np.sum(exp_x, axis = 1 , keepdims=True)

    def __sigmoid_grad(self,x):
        return x * (1-x)  

    def next(self , X) :
        X = np.hstack([np.ones((X.shape[0] , 1)),X ])    
        self.net_grad = np.dot(X, self.w)
        self.O = self.output_function(self.net_grad)
        return self.O
    
    def _prev(self, y_true , O_down)  :
        new_grad = self.O - y_true
        dw = np.dot(np.hstack([np.ones((O_down.shape[0], 1)), O_down]).T , new_grad) / y_true.shape[0]
        d_O_down = np.dot(new_grad , self.w.T)
        dO_prev =   d_O_down[:,1:]
        self.w = self.w - self.lr * dw
        return dO_prev
    
class NeuralNetwork:
    def __init__(self, M = 32 , n = 3072 , HiddenLayer = [512 , 265 ]  , target_class = 36 , lr = 0.01):
        self.lr = lr
        self.layers = []
        self.Batch_size = M
        prev_dimn = n
        for h in HiddenLayer:
            layer = Layer(prev_dimn , h , is_sigmoid = True , learning_rate = self.lr)
            self.layers.append(layer)
            prev_dimn = h
        self.output_layer = FinalLayer(prev_dimn , target_class , is_softmax = True , lr = self.lr)

    def next(self,X) : 
        O_down = X 
        for layer in self.layers :
            O_down = layer.next(O_down)

        predictions = self.output_layer.next(O_down)
        return predictions
    
    def corss_entropy_loss(self, y_true , y_pred):
        m = y_true.shape[0]
        return - np.sum(y_true * np.log(y_pred + 1e-10 )) / m
     
    def _prev(self,Y , X) : 
        O = X 
        stored_outputs = [X]
        for layer in self.layers :
            O = layer.next(O)
            stored_outputs.append(O)
        predictions = self.output_layer.next(O)

       

        do_down = self.output_layer._prev(Y , stored_outputs[-1])
        for i in range(len(self.layers)-1 , -1 , -1):
            do_down = self.layers[i]._prev(do_down , stored_outputs[i])

        return predictions  

    def predict(self,X) :
        predictions = self.next(X)
        return np.argmax(predictions , axis = 1)   
    
    def fit(self , X , Y , epochs = 10 ):
        for epoch in range(epochs) : 
            comb = np.random.permutation(X.shape[0])
            X_shuffled = X[comb]
            Y_shuffled = Y[comb]
            J = 0

            for i in range(0, X.shape[0] , self.Batch_size):
                X_batch = X_shuffled[i : i + self.Batch_size]
                Y_batch = Y_shuffled[i : i + self.Batch_size]
                predictions = self._prev(Y_batch , X_batch)
                loss = self.corss_entropy_loss(Y_batch , predictions)
                J += loss
           
            print(f"Epoch {epoch + 1} / {epochs} , Loss : {J}")

               


In [4]:
hidden_layer_length = [[512],[512,256],[512,256,128],[512,256,128,64],]

for h_len in hidden_layer_length:
    model = NeuralNetwork(M = 32 , n = 3072 , HiddenLayer = h_len  , target_class = 36 , lr = 0.01) 
    model.fit(X_train , y_train , epochs = 20)
    y_test_prediction = model.predict(X_test)
    y_train_prediction = model.predict(X_train)
    print("Train Accuracy: ", np.mean(y_train_prediction == np.argmax(y_train , axis = 1)) * 100)
    print("Test Accuracy: ", np.mean(y_test_prediction == np.argmax(y_test , axis = 1)) * 100)
    
    def results_metrics(y_true, y_pred):
        f1 = f1_score(np.argmax(y_true , axis = 1) , y_pred , average='macro')
        precision = f1_score(np.argmax(y_true , axis = 1) , y_pred , average='macro', zero_division=0)
        recall = f1_score(np.argmax(y_true , axis = 1) , y_pred , average='macro', zero_division=0)
        return f1, precision, recall

    f1, precision, recall = results_metrics(y_test, y_test_prediction)
    print(f"f1 Score for hidden layer size {h_len} : ", f1)
    print(f"Precision for hidden layer size {h_len} : ", precision)
    print(f"Recall for hidden layer size {h_len} : ", recall)
    print("---------------------------------------------------")

    f1, precision, recall = results_metrics(y_train, y_train_prediction)
    print(f"Train f1 Score for hidden layer size {h_len} : ", f1)
    print(f"Train Precision for hidden layer size {h_len} : ", precision)
    print(f"Train Recall for hidden layer size {h_len} : ", recall)
    print("===================================================")

Epoch 1 / 20 , Loss : 2169.539596130592
Epoch 2 / 20 , Loss : 1709.0519445343086
Epoch 3 / 20 , Loss : 1409.3300034000856
Epoch 4 / 20 , Loss : 1227.7618031046363
Epoch 5 / 20 , Loss : 1113.3505808659609
Epoch 6 / 20 , Loss : 1035.6529618051557
Epoch 7 / 20 , Loss : 979.3547647713399
Epoch 8 / 20 , Loss : 936.8484610928751
Epoch 9 / 20 , Loss : 902.8982841636717
Epoch 10 / 20 , Loss : 875.4175325984816
Epoch 11 / 20 , Loss : 852.1567037173903
Epoch 12 / 20 , Loss : 831.8331640480155
Epoch 13 / 20 , Loss : 814.5035089393419
Epoch 14 / 20 , Loss : 798.9628084071877
Epoch 15 / 20 , Loss : 784.8655204732895
Epoch 16 / 20 , Loss : 771.3079723340414
Epoch 17 / 20 , Loss : 759.6291988057023
Epoch 18 / 20 , Loss : 748.6184164177948
Epoch 19 / 20 , Loss : 738.2549367152521
Epoch 20 / 20 , Loss : 728.6544648062104
Train Accuracy:  72.42592592592592
Test Accuracy:  69.5
f1 Score for hidden layer size [512] :  0.6941844940930983
Precision for hidden layer size [512] :  0.6941844940930983
Recall fo

In [4]:
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

hidden_layer_length = [
    [512],
    [512, 256],
    [512, 256, 128],
    [512, 256, 128, 64]
]

for h_len in hidden_layer_length:
    print(f"\n=== Hidden layer architecture: {h_len} ===")

    model = NeuralNetwork(M=32, n=3072, HiddenLayer=h_len, target_class=36, lr=0.01)
    model.fit(X_train, y_train, epochs=20)

    # Predictions
    y_test_prediction = model.predict(X_test)
    y_train_prediction = model.predict(X_train)

    # Accuracy
    print("\nAccuracy:")
    print(f"Train Accuracy: {np.mean(y_train_prediction == np.argmax(y_train, axis=1)) * 100:.2f}%")
    print(f"Test Accuracy: {np.mean(y_test_prediction == np.argmax(y_test, axis=1)) * 100:.2f}%")

    # ---------- Metric Calculation ----------
    def results_metrics(y_true, y_pred):
        y_true_labels = np.argmax(y_true, axis=1)
        f1 = f1_score(y_true_labels, y_pred, average=None)
        precision = precision_score(y_true_labels, y_pred, average=None, zero_division=0)
        recall = recall_score(y_true_labels, y_pred, average=None, zero_division=0)
        return f1, precision, recall

    # ---------- TEST SET ----------
    f1_test, precision_test, recall_test = results_metrics(y_test, y_test_prediction)
    results_test = pd.DataFrame({
        "Class": np.arange(1, len(f1_test) + 1),
        "F1": f1_test,
        "Precision": precision_test,
        "Recall": recall_test
    })

    print("\nPer-Class Metrics (Test):")
    print(results_test.round(4).to_string(index=False))

    print("\nAverage (Macro) Metrics (Test):")
    print(f"F1: {np.mean(f1_test):.4f}, Precision: {np.mean(precision_test):.4f}, Recall: {np.mean(recall_test):.4f}")
    print("---------------------------------------------------")

    # ---------- TRAIN SET ----------
    f1_train, precision_train, recall_train = results_metrics(y_train, y_train_prediction)
    results_train = pd.DataFrame({
        "Class": np.arange(1, len(f1_train) + 1),
        "F1": f1_train,
        "Precision": precision_train,
        "Recall": recall_train
    })

    print("\nPer-Class Metrics (Train):")
    print(results_train.round(4).to_string(index=False))

    print("\nAverage (Macro) Metrics (Train):")
    print(f"F1: {np.mean(f1_train):.4f}, Precision: {np.mean(precision_train):.4f}, Recall: {np.mean(recall_train):.4f}")
    print("===================================================")



=== Hidden layer architecture: [512] ===
Epoch 1 / 20 , Loss : 2164.1136174359053
Epoch 2 / 20 , Loss : 1711.9924094252192
Epoch 3 / 20 , Loss : 1414.111750329867
Epoch 4 / 20 , Loss : 1231.2960954362711
Epoch 5 / 20 , Loss : 1115.8349969274125
Epoch 6 / 20 , Loss : 1037.164340707464
Epoch 7 / 20 , Loss : 980.6332803195786
Epoch 8 / 20 , Loss : 937.7839732793944
Epoch 9 / 20 , Loss : 903.459233938744
Epoch 10 / 20 , Loss : 875.709899595438
Epoch 11 / 20 , Loss : 852.3532527347066
Epoch 12 / 20 , Loss : 832.2435396461796
Epoch 13 / 20 , Loss : 814.693108325888
Epoch 14 / 20 , Loss : 799.1164898660185
Epoch 15 / 20 , Loss : 785.2315817981989
Epoch 16 / 20 , Loss : 771.8034040728877
Epoch 17 / 20 , Loss : 760.2397868031684
Epoch 18 / 20 , Loss : 748.9250420419403
Epoch 19 / 20 , Loss : 738.8023382491006
Epoch 20 / 20 , Loss : 728.6686700413475

Accuracy:
Train Accuracy: 72.20%
Test Accuracy: 69.52%

Per-Class Metrics (Test):
 Class     F1  Precision  Recall
     1 0.7809     0.7720  0.79

In [7]:
h_len = [512]
print(f"\n=== Hidden layer architecture: {h_len} ===")

model = NeuralNetwork(M=32, n=3072, HiddenLayer=h_len, target_class=36, lr=0.01)
model.fit(X_train, y_train, epochs=200)

# Predictions
y_test_prediction = model.predict(X_test)
y_train_prediction = model.predict(X_train)

# Accuracy
print("\nAccuracy:")
print(f"Train Accuracy: {np.mean(y_train_prediction == np.argmax(y_train, axis=1)) * 100:.2f}%")
print(f"Test Accuracy: {np.mean(y_test_prediction == np.argmax(y_test, axis=1)) * 100:.2f}%")

# ---------- Metric Calculation ----------
def results_metrics(y_true, y_pred):
    y_true_labels = np.argmax(y_true, axis=1)
    f1 = f1_score(y_true_labels, y_pred, average=None)
    precision = precision_score(y_true_labels, y_pred, average=None, zero_division=0)
    recall = recall_score(y_true_labels, y_pred, average=None, zero_division=0)
    return f1, precision, recall

# ---------- TEST SET ----------
f1_test, precision_test, recall_test = results_metrics(y_test, y_test_prediction)
results_test = pd.DataFrame({
    "Class": np.arange(1, len(f1_test) + 1),
    "F1": f1_test,
    "Precision": precision_test,
    "Recall": recall_test
})

print("\nPer-Class Metrics (Test):")
print(results_test.round(4).to_string(index=False))

print("\nAverage (Macro) Metrics (Test):")
print(f"F1: {np.mean(f1_test):.4f}, Precision: {np.mean(precision_test):.4f}, Recall: {np.mean(recall_test):.4f}")
print("---------------------------------------------------")

# ---------- TRAIN SET ----------
f1_train, precision_train, recall_train = results_metrics(y_train, y_train_prediction)
results_train = pd.DataFrame({
    "Class": np.arange(1, len(f1_train) + 1),
    "F1": f1_train,
    "Precision": precision_train,
    "Recall": recall_train
})

print("\nPer-Class Metrics (Train):")
print(results_train.round(4).to_string(index=False))

print("\nAverage (Macro) Metrics (Train):")
print(f"F1: {np.mean(f1_train):.4f}, Precision: {np.mean(precision_train):.4f}, Recall: {np.mean(recall_train):.4f}")
print("===================================================")


=== Hidden layer architecture: [512] ===
Epoch 1 / 200 , Loss : 2165.4585596611087


KeyboardInterrupt: 